In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/1.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,akjmXXq,0,FEM,28.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,MV6b6R9,0,FEM,30.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,zvxv1Rm,0,FEM,22.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,8oqJzL,0,MASC,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ez533B,0,MASC,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12332167


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
395895,1112369306,6rP8A9r,1,MASC,35.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
5454125,1112396113,mzda94E,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,NaN
5401233,1112357032,kP0ZjaQ,1,MASC,26.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,NaN
11184290,1112373477,1bZB8L,0,MASC,35.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
11579210,1112337804,96zewX9,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2220979,1112305215,96LqQ0w,1,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
3528669,1112381105,2zPD4Ax,1,FEM,20.0,-,-,-,-,-,...,False,True,False,False,False,False,False,True,False,NaN
243285,1112260584,W9Wm80e,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
6402860,1111305364,Z302Vz,0,FEM,26.0,-,-,-,En Curso,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
1588988,1112457520,lDLlY03,1,MASC,18.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,1.0


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
4664890,1112378617,96lkPBP,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
8184428,1112383417,zvxWqb3,0,MASC,22.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
12104610,1112426253,ZpEKl8,0,MASC,41.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.0
5559299,1112245098,xk9AqN6,1,MASC,32.0,-,-,-,-,Graduado,...,True,True,False,False,False,False,False,False,False,0.0
8036523,1112357771,zv8p3Jk,0,FEM,25.0,-,-,-,-,-,...,True,False,False,False,False,False,False,False,False,0.0
2689515,1112408026,MVORvdo,1,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,True,False,0.0
11371691,1112338147,8Wokmx,0,FEM,29.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,True,False,False,False,0.0
8896776,1112423974,8MZkb6L,0,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
488702,1112284410,RzML4A6,1,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.0
10282171,1112375096,8jpkeL,0,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  11098950 Test:  1233217


In [12]:
import gc
gc.collect()

379

In [ ]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=4096,max_depth=13,n_estimators=500,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

72.63060758974292


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV2.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV2.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
81516,29246,1112369774,6Aapjv,MASC,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
15787,67772,1112460415,EPombz,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
98480,98330,1112505801,EAYO09,MASC,39.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
23799,70695,1112461848,GNd5P29,MASC,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
39114,54069,1112445738,LNPV9MB,MASC,43.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,True,False,0.0
14110,61618,1112455550,6538Xv,FEM,40.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
75463,42253,1112423251,YqOv1,MASC,60.0,-,-,-,Graduado,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
91539,90387,1112467498,8YkQ9x,MASC,37.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
4975,82862,1112466289,KBrEOOX,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
5652,30463,1112377689,NzP846J,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.55696771, 0.44303229],
       [0.58494284, 0.41505716],
       [0.59153026, 0.40846974],
       ...,
       [0.84154186, 0.15845814],
       [0.57794176, 0.42205824],
       [0.91297643, 0.08702357]])

In [18]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [19]:
prediccion['sepostulo']=y_final2

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.443032
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.415057
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.408470
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.330277
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.350513


In [21]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.443032
1,1,0.415057
2,2,0.408470
3,3,0.330277
4,4,0.350513


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion15.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.5891178732935961